In [5]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Recommender_System/MPU_GEO.csv')

ModuleNotFoundError: No module named 'google'

In [6]:
dataset.head()

NameError: name 'dataset' is not defined

In [ ]:
#renaming our dataset
dataset.columns = ['user_id', 'item_id', 'rating', 'timestamp' ]


In [ ]:
 # detecting missing values
missing_values = dataset.isnull().sum()

In [ ]:
# plotting missing values
missing_values.plot.barh()
plt.xlabel(\"No of missing values\")
plt.title(\"Missing Values\")

In [ ]:
# Filling missing values with mean
dataset['rating'].fillna(dataset['rating'].mean(), inplace=True)

In [ ]:
# checking the dataset
dataset.isnull().sum()

In [ ]:
# total number of user, item and rating and timestamp
dataset.count()

In [ ]:
uniq = []
# total number of unique users
uniq.append(dataset.user_id.unique().size)
uniq.append(dataset.item_id.unique().size)
uniq

In [ ]:
# We have 643951 unique user and 18306 unique item. Which means a single custer has rated multiple product.
# which is the key idea we need build our 
# recommender system.

# Now let's visualize it 
lev = ['user', 'item']
plt.barh(lev, uniq)
plt.xlabel('Number of Uniqueness')
plt.title('Unique Customer and Item')

In [ ]:
rat = dataset.sort_values('rating')

In [ ]:
rat.head()

In [ ]:
plt.barh(rat['user_id'].head(1000), rat['rating'].head(1000))
plt.show()

In [ ]:
# Number of count for each unique item
item_count= dataset.item_id.value_counts()

In [ ]:
# box plot for item count
item_count.plot.box()
plt.ylabel('No of count')

In [ ]:
# pie chart for firt 50 unique item
item_count.head(50).plot.pie()
plt.show()

In [ ]:
# pie chart for firt 50 unique item
item_count.plot.area()
plt.show()

In [ ]:
# number of unique rating
rating_count = dataset.rating.value_counts()
rating_count.head()

In [ ]:
# It's clear that most of the user rated 3 

# Let's visualize it
rating_count.plot.area()

In [ ]:
# Number of unique user count
user_count = dataset.user_id.value_counts()
user_count.head()
user_count.plot.box()

In [ ]:
user_count.plot.line()

In [ ]:
user_count.head(100).plot.pie()

In [ ]:
#Statistical summary
dataset.describe()

In [ ]:
dataset.head()

In [ ]:
#Converting users and items to categories
dataset.user_id = dataset.user_id.astype('category').cat.codes.values
dataset.item_id = dataset.item_id.astype('category').cat.codes.values

In [ ]:
#Create training and testing dataset
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size=0.2)

In [ ]:
#Import libraries for deep learning
import keras
from IPython.display import SVG
from keras.optimizers import Adam
from keras.utils.vis_utils import model_to_dot

n_users, n_items = len(dataset.user_id.unique()), len(dataset.item_id.unique())
n_latent_factors = 3

In [ ]:
#Create embeddings and format input
item_input = keras.layers.Input(shape=[1],name='Item')
item_embedding = keras.layers.Embedding(n_items + 1, n_latent_factors, name='Item-Embedding')(item_input)
item_vec = keras.layers.Flatten(name='FlattenItems')(item_embedding)

user_input = keras.layers.Input(shape=[1],name='User')
user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users + 1, n_latent_factors,name='User-Embedding')(user_input))

prod = keras.layers.dot([item_vec, user_vec], axes = 1,name='DotProduct')
model = keras.Model([user_input, item_input], prod)
model.compile('adam', 'mean_squared_error')

In [ ]:
#Showing the architecture of machine learning model
SVG(model_to_dot(model,  show_shapes=True, show_layer_names=True, rankdir='HB').create(prog='dot', format='svg'))

In [ ]:
#Model summary and learning parameters
model.summary()

In [ ]:
history = model.fit([train.user_id, train.item_id], train.rating, epochs=100, verbose=0)


In [ ]:
#Same procedure but now with neural networks
pd.Series(history.history['loss']).plot(logy=True)
plt.xlabel("Epoch")
plt.ylabel("Train Error")


In [ ]:
y_hat = np.round(model.predict([test.user_id, test.item_id]),0)
y_true = test.rating

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_true, y_hat)

In [ ]:
item_embedding_learnt = model.get_layer(name='Item-Embedding').get_weights()[0]
pd.DataFrame(item_embedding_learnt).describe()

In [ ]:
user_embedding_learnt = model.get_layer(name='User-Embedding').get_weights()[0]
pd.DataFrame(user_embedding_learnt).describe()

In [ ]:
n_latent_factors_user = 5
n_latent_factors_item = 8

item_input = keras.layers.Input(shape=[1],name='Item')
item_embedding = keras.layers.Embedding(n_items + 1, n_latent_factors_item, name='Item-Embedding')(item_input)
item_vec = keras.layers.Flatten(name='FlattenItems')(item_embedding)
item_vec = keras.layers.Dropout(0.2)(item_vec)


user_input = keras.layers.Input(shape=[1],name='User')
user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users + 1, n_latent_factors_user,name='User-Embedding')(user_input))
user_vec = keras.layers.Dropout(0.2)(user_vec)


concat = keras.layers.concatenate([item_vec, user_vec],name='Concat')
concat_dropout = keras.layers.Dropout(0.2)(concat)
dense = keras.layers.Dense(200,name='FullyConnected')(concat)
dropout_1 = keras.layers.Dropout(0.2,name='Dropout')(dense)
dense_2 = keras.layers.Dense(100,name='FullyConnected-1')(concat)
dropout_2 = keras.layers.Dropout(0.2,name='Dropout')(dense_2)
dense_3 = keras.layers.Dense(50,name='FullyConnected-2')(dense_2)
dropout_3 = keras.layers.Dropout(0.2,name='Dropout')(dense_3)
dense_4 = keras.layers.Dense(20,name='FullyConnected-3', activation='relu')(dense_3)


result = keras.layers.Dense(1, activation='relu',name='Activation')(dense_4)
adam = Adam(lr=0.005)
model = keras.Model([user_input, item_input], result)
model.compile(optimizer=adam,loss= 'mean_absolute_error')

In [ ]:
SVG(model_to_dot(model,  show_shapes=True, show_layer_names=True, rankdir='HB').create(prog='dot', format='svg'))

In [ ]:
model.summary()

In [ ]:
history = model.fit([train.user_id, train.item_id], train.rating, epochs=250, verbose=0)

In [ ]:
#Results
y_hat_2 = np.round(model.predict([test.user_id, test.item_id]),0)
print(mean_absolute_error(y_true, y_hat_2))

print(mean_absolute_error(y_true, model.predict([test.user_id, test.item_id])))